# Team formation for HW2

The idea behind our mechanism was to include preferred meeting times in the optimization model from the lecture. 
We divided the day into two groups: day (1) and evening (2). For the evening not to have a higher weight than the morning (the data is categorical nominal data), we chose to minimize the maximum difference in preferred meeting times between two team members in each team. The day can be devided into more groups, but the problem very quickly becomes very difficult to solve when there is a lot of students and a lot of possible meeting times.
To still make sure the groups are even, constraints about the theory score and coding score are added to the model.
 
An excel file as a csv file from each student should be saved in a folder. Each excel file should have one row and three columns, where the student's coding score, theory score and preferred meeting time is specified.
For the purpose of testing the model randomized data is used. 

Importing the packages

In [1]:
import os
import pandas as pd
import csv
import glob
import numpy as np
os.getcwd()
import math # for ceil

%pip install -q gurobipy
import gurobipy as gp

Note: you may need to restart the kernel to use updated packages.


### Data
Code for loading in the responses from each student

In [3]:
# Load in the csv files
path = r'C:\Users\Laura\Dropbox\Skole\DTU\Udveksling\Arrival\School\Advanced Data Analytics\Responses'

filenames = glob.glob(path + "/*.csv")

list_of_responses = []
for filename in filenames:
    d = pd.read_csv(filename, index_col=None,header=None,sep=';')
    list_of_responses.append(d)
    
categories = ["Coding", "Theory", "Meeting times"]    

comfort_scores = np.array(list_of_responses).ravel().reshape(len(list_of_responses), len(categories))

NUM_STUDENTS = len(comfort_scores)

students = [i for i in range(0, NUM_STUDENTS)]

comfort = {}
for i in range(0, NUM_STUDENTS):
    for j in range(0, len(categories)):
        comfort[(i, categories[j])] = comfort_scores[i][j]

  
NUM_TEAMS = math.ceil(len(students) // 3)

df = pd.DataFrame(
    # data = np.random.randint(1, 6, size=(NUM_STUDENTS, len(categories))),
    # use the dictionary of comfort values
    data = [[comfort[(i,j)] for j in categories] for i in students],
    columns=categories,
    index=students,
)

display(df)

,Coding,Theory,Meeting times
0,1,2,2
1,2,5,1
2,4,2,2
3,2,3,1
4,5,2,2
5,1,5,2


### Random data for testing the model

In [49]:
# Generate a set of students with comfort (on a scale of 1 to 5) in three different categories: "Coding", "Theory", and "Writing"
np.random.seed(42)
NUM_STUDENTS = 37
categories = ["Coding", "Theory", "Meeting times"]
students = [i for i in range(0, NUM_STUDENTS)]

# Generate a random comfort score for each student in each category
comfort_scores = np.random.randint(1, 6, size=(NUM_STUDENTS, len(categories[0:2])))
meeting_scores = np.random.randint(1, 3, size=(NUM_STUDENTS, 1))

# Gather the scores to one array
comfort_scores = np.c_[comfort_scores, meeting_scores]



# Dictionary of the comfort of each student in each category
comfort = {}
for i in range(0, NUM_STUDENTS):
    for j in range(0, len(categories)):
        comfort[(i, categories[j])] = comfort_scores[i][j]  
 
    

df = pd.DataFrame(
    data = [[comfort[(i,j)] for j in categories] for i in students],
    columns=categories,
    index=students,
)

display(df)

NUM_TEAMS = len(students)//2

,Coding,Theory,Meeting times
0,4,5,1
1,3,5,1
2,5,2,2
3,3,3,2
4,3,5,1
5,4,3,2
6,5,2,2
7,4,2,2
8,4,5,1
9,1,4,1


### Linear Programming model

In [66]:
m = gp.Model("Team formation")

# Variables
x = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="x")
y = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="y")
t = m.addVars(range(NUM_TEAMS),lb = 0, name="t")
v = m.addVars(students, students, vtype=gp.GRB.BINARY, name = "v")


# Objective function
m.setObjective(gp.quicksum(t[j] for j in range(NUM_TEAMS)), gp.GRB.MINIMIZE)


# Constraints
# Each student is in exactly one team
m.addConstrs(gp.quicksum(x[i,j] for j in range(NUM_TEAMS)) == 1 for i in students)

# Each team has at least 2 students if the team exists
m.addConstrs(gp.quicksum(x[i,j] for i in students) >= 2*y[j] for j in range(NUM_TEAMS)) 

# Each team has at most 3 students if the team exists
m.addConstrs(gp.quicksum(x[i,j] for i in students) <= 3*y[j] for j in range(NUM_TEAMS))


# If two students are on the same team, then v = 1
M = 2
m.addConstrs(x[i,j] >= x[k,j] - M*(1-v[i,k]) for i in students for k in students for j in range(NUM_TEAMS))


# Constraint ensuring v is not 0 every time
m.addConstrs(gp.quicksum(v[i,k] for k in students) >= 2 for i in students)


# t[j] is the maximum difference between two team members in each team. 
MM = 3
m.addConstrs(df.at[i,"Meeting times"] * x[i,j] - df.at[k,"Meeting times"] * x[k,j] <= t[j]+MM*(1-v[i,k]) for i in students for k in students for j in range(NUM_TEAMS)) 
m.addConstrs(-(df.at[i,"Meeting times"] * x[i,j] - df.at[k,"Meeting times"] * x[k,j]) <= t[j]+MM*(1-v[i,k]) for i in students for k in students for j in range(NUM_TEAMS)) 


# Ensuring a minimum coding score of 7 in each team
MIN_CODING_SCORE = 6
m.addConstrs(gp.quicksum(comfort[(i,categories[0])] * x[i,j] for i in students) >= MIN_CODING_SCORE * y[j] for j in range(NUM_TEAMS))


# Ensuring a minimum theory score of 5 in each team
MIN_THEORY_SCORE = 5
m.addConstrs(gp.quicksum(comfort[(i,categories[1])] * x[i,j] for i in students) >= MIN_THEORY_SCORE * y[j] for j in range(NUM_TEAMS))


# Total number of teams is ceil(number of students / 3)
m.addConstr(gp.quicksum([y[j] for j in range(NUM_TEAMS)]) == math.ceil(len(students) / 3))


<gurobi.Constr *Awaiting Model Update*>

In [67]:
m.write('team-formation.lp')

In [70]:
# Function that takes in a Gurobi model and returns a list of teams, where each team is a list of students
def get_teams(m):
    teams = []
    for j in range(NUM_TEAMS):
        if m.getVarByName('y[' + str(j) + ']').x > 0.5:
            team = []
            for i in range(NUM_STUDENTS):
                if m.getVarByName('x[' + str(i) + ',' + str(j) + ']').x > 0.5:
                    team.append(i)
            teams.append(team)
    return teams
    
# Function that takes as input a list of teams and returns a table with team number, team members, total comfort, and average comfort in each category, with each category in a separate column
def analyze_teams(teams):
    # Compute num_two_student_teams, num_three_student_teams, avg_total_comfort, avg_comfort_by_category, min_total_comfort, max_total_comfort, min_comfort, max_comfort, total_comfort_sum, total_comfort_sum_sq, comfort_sums, comfort_sum_sqs
    min_total_comfort = float('inf')
    max_total_comfort = float('-inf')
    min_comfort = [float('inf')] * len(categories[0:2])
    max_comfort = [float('-inf')] * len(categories[0:2])
    max_timediff = float('-inf')
    

    # Track standard deviation of total comfort
    total_comfort_sum = 0
    total_comfort_sum_sq = 0

    # Track standard deviation of scores in each category
    comfort_sums = [0] * len(categories[0:2])
    comfort_sum_sqs = [0] * len(categories[0:2])

    # Track how many teams have two students and how many have three students
    num_two_student_teams = 0
    num_three_student_teams = 0

    # Track average total comfort across all teams and average comfort by category
    avg_total_comfort = 0
    avg_comfort_by_category = [0] * len(categories[0:2])

    for j in range(len(teams)):
        # Current team
        curr_team = teams[j]

        # Compute comfort of the team
        curr_comfort_by_category = [gp.quicksum(comfort[(i,c)] for i in curr_team).getValue() for c in categories[0:2]]
        total_comfort = sum(curr_comfort_by_category)
        
        time = [df.at[i,"Meeting times"] for i in curr_team]
        time_diff = max(time)-min(time)

        # Update min_total_comfort, max_total_comfort, total_comfort_sum, total_comfort_sum_sq
        if total_comfort < min_total_comfort:
            min_total_comfort = total_comfort
        if total_comfort > max_total_comfort:
            max_total_comfort = total_comfort
        if time_diff > max_timediff:
            max_timediff = time_diff
        total_comfort_sum += total_comfort
        total_comfort_sum_sq += total_comfort**2

        # Update min_comfort, max_comfort, comfort_sums, comfort_sum_sqs
        for k in range(len(categories[0:2])):
            if curr_comfort_by_category[k] < min_comfort[k]:
                min_comfort[k] = curr_comfort_by_category[k]
            if curr_comfort_by_category[k] > max_comfort[k]:
                max_comfort[k] = curr_comfort_by_category[k]
            comfort_sums[k] += curr_comfort_by_category[k]
            comfort_sum_sqs[k] += curr_comfort_by_category[k]**2

        # Update avg_total_comfort, avg_comfort_by_category
        avg_total_comfort += total_comfort
        for k in range(len(categories[0:2])):
            avg_comfort_by_category[k] += curr_comfort_by_category[k]

        # Update num_two_student_teams, num_three_student_teams
        if len(curr_team) == 2:
            num_two_student_teams += 1
        else:
            num_three_student_teams += 1

    # Print out the minimum/maximum/stddev total and minimum score in each category across all teams
    print("\n####################")
    print("Total number of teams:", num_two_student_teams + num_three_student_teams)
    print("Number of two-student teams:", num_two_student_teams)
    print("Number of three-student teams:", num_three_student_teams)
    print('The minimum total score across all teams is', min_total_comfort)
    print('The maximum total score across all teams is', max_total_comfort)
    print('The standard deviation of the total score across all teams is', (total_comfort_sum_sq - (total_comfort_sum**2)/NUM_TEAMS)/(NUM_TEAMS-1))
    print('The minimum score in each category across all teams is', min_comfort)
    print('The maximum score in each category across all teams is', max_comfort)
    print('The maximum time diff across all teams is', max_timediff)
    print('The standard deviation of the scores in each category across all teams is', [(comfort_sum_sqs[k] - (comfort_sums[k]**2)/NUM_TEAMS)/(NUM_TEAMS-1) for k in range(len(categories[0:2]))])

    
    print("\n####################")
    print("Team number".ljust(13), "Team members".ljust(15), "Comfort".ljust(8), " ".join([category.ljust(8) for category in categories[0:2]]), "Difference in meeting times".ljust(6))
    for j in range(len(teams)):
        curr_team = teams[j]
        curr_comfort_by_category = [gp.quicksum(comfort[(i,c)] for i in curr_team).getValue() for c in categories[0:2]]
        curr_total_comfort = sum(curr_comfort_by_category)
        
        time = [df.at[i,"Meeting times"] for i in curr_team]
        time_diff = max(time)-min(time)
        
        print(
            str(j).ljust(13), 
            str([i for i in curr_team]).ljust(15), 
            # sum comfort of all students in the team
            str(int(curr_total_comfort)).ljust(8),
            " ".join([str(int(curr_comfort_by_category[k])).ljust(8) for k in range(len(categories[0:2]))]),
             str(int(time_diff)).ljust(6))
            

    # Print the average, min, max, standard deviation across the columns Total Comfort and each category
    # Format the standard deviation to only display 1 decimal place
    print("Average".ljust(13), " ".ljust(15),
        str(round(avg_total_comfort/(num_two_student_teams + num_three_student_teams), 2)).ljust(8),
        " ".join([str(round(avg_comfort_by_category[k]/(num_two_student_teams + num_three_student_teams), 2)).ljust(8) for k in range(len(categories[0:2]))]))
    print("Min".ljust(13), " ".ljust(15),
        str(int(min_total_comfort)).ljust(8),
        " ".join([str(int(min_comfort[k])).ljust(8) for k in range(len(categories[0:2]))]))
    print("Max".ljust(13), " ".ljust(15),
        str(int(max_total_comfort)).ljust(8),
        " ".join([str(int(max_comfort[k])).ljust(8) for k in range(len(categories[0:2]))]))
    print("Std Dev".ljust(13), " ".ljust(15),
        str(round((total_comfort_sum_sq - (total_comfort_sum**2)/NUM_TEAMS)/(NUM_TEAMS-1), 2)).ljust(8),
        " ".join([str(round((comfort_sum_sqs[k] - (comfort_sums[k]**2)/NUM_TEAMS)/(NUM_TEAMS-1), 2)).ljust(8) for k in range(len(categories[0:2]))]))

# Analyze model
def analyze_model(m):
    # Check if the model is optimal
    if m.status != gp.GRB.Status.OPTIMAL:
        print('Optimization was stopped with status %d' % m.status)
        # If infeasible, compute and display IIS
        if m.status == gp.GRB.Status.INFEASIBLE:
            m.computeIIS()
            m.write('team-formation.ilp')
    else:
        # Print the objective value
        # If optimal, analyze the solution
        print('The objective value is', m.objVal)
        
        # Get teams
        teams = get_teams(m)
        analyze_teams(teams)

In [71]:
categories = ["Coding", "Theory", "Meeting"] 

m.setObjective(gp.quicksum(t[j] for j in range(NUM_TEAMS)), gp.GRB.MINIMIZE)


# Silence the output
m.setParam('OutputFlag', False)

# Optimize and analyze the output
m.optimize()

analyze_model(m)

The objective value is 0.0

####################
Total number of teams: 13
Number of two-student teams: 2
Number of three-student teams: 11
The minimum total score across all teams is 13.0
The maximum total score across all teams is 23.0
The standard deviation of the total score across all teams is 76.69281045751634
The minimum score in each category across all teams is [6.0, 5.0]
The maximum score in each category across all teams is [12.0, 13.0]
The maximum time diff across all teams is 0
The standard deviation of the scores in each category across all teams is [19.790849673202615, 21.3202614379085]

####################
Team number   Team members    Comfort  Coding   Theory   Difference in meeting times
0             [21, 32, 36]    19       11       8        0     
1             [8, 18, 22]     20       9        11       0     
2             [7, 26]         15       8        7        0     
3             [2, 17, 34]     23       12       11       0     
4             [25, 31, 35]  